# Initialize cluster and Moving randomly any molecule
# avoiding that any atoms overlaps each other:

In [1]:
# crating the path (PYTHONPATH) to our module.
# assuming that our 'amcess' directory is out ('..') of our current directory 
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
def visualize(
        xyz_file: str, 
        sphere_radius: int, 
        sphere_center: tuple,
        style='sphere',
        ):
        """
        Visualize the molecule with a sphere around the molecule.
        """

        sr = sphere_radius
        cx, cy, cz = sphere_center

        if not xyz_file:
                return None

        # starting visualization
        xyz_view = py3Dmol.view() # width=400, height=300)#, linked=False, viewergrid=(2,2))
        xyz_view.addModelsAsFrames(xyz_file,'xyz')
        # xyz_view.setStyle({style: {'radius': 0.6}})

        if style == 'stick':
                xyz_view.setStyle({'stick': {'radius': 0.15}})
        else:
                xyz_view.setStyle({'sphere': {'radius': 0.8}})


        xyz_view.zoomTo()

        # animation base on several XYZ coordinates snapshots
        xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 1})

        # sphere center
        sc = {"x": cx, "y": cy, "z": cz}

        # 20% to check visually that everything is inside
        xyz_view.addSphere({'center': sc, 
                'radius': sr, 
                'color' :'yellow',
                'alpha': 0.5,
                })

        # cartesian 3D axes
        x_axis = {'start': {'x': cx-sr, 'y':cy, 'z':cz}, 'end': {'x': cx+sr, 'y':cy, 'z':cz}}
        y_axis = {'start': {'x': cx, 'y':cy-sr, 'z':cz}, 'end': {'x': cx, 'y':cy+sr, 'z':cz}}
        z_axis = {'start': {'x': cx, 'y':cy, 'z':cz-sr}, 'end': {'x': cx, 'y':cy, 'z':cz+sr}}

        xyz_view.addLine(x_axis)
        xyz_view.addLine(y_axis)
        xyz_view.addLine(z_axis)

        xyz_view.addLabel("x", {
                'position':x_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })
        xyz_view.addLabel("y", {
                'position':y_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })
        xyz_view.addLabel("z", {
                'position':z_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })

        return xyz_view

# xyz_view.show()

In [3]:
# py3Dmol: a simple IPython/Jupyter widget to embed an interactive 
# 3Dmol.js viewer in a notebook.
import py3Dmol

# importing de Molecule Class
from amcess.molecule import Molecule
from amcess.cluster import Cluster

In [5]:
hf = {"atoms": [("H", 0, 0, 0), ("F", 1, 0, 0)]}
cc = {"atoms": [("C", 0, 0, 0), ("C", 1.2, 0, 0)]}
ab = Cluster(hf, cc)

In [7]:
xyz_data = ab.GetBlockXYZ().replace('\t', '').split('\n')
xyz_data = xyz_data[2:]
xyz_data = ';'.join(xyz_data)

print(xyz_data)

H          0.00000000     0.00000000     0.00000000;F          1.00000000     0.00000000     0.00000000;C          0.00000000     0.00000000     0.00000000;C          1.20000000     0.00000000     0.00000000;


In [9]:
ab_xyz = ""

ab.SetSphereR(4)
# ab.freeze_molecule = 0

for _ in range(100):
    mol = ab.GetRandomGen().choice(ab.GetTotalMol())

    ab = ab.MoveMol(
        molecule=mol,
        max_step=3,
        max_rotation=10,
        max_closeness=0.5,
    )
    # print(ab.xyz)
    ab_xyz += ab.GetBlockXYZ()

# file_name = "cluster_test.xyz"
# with open(file_name, "w") as f:
#     f.write(ab_xyz)

In [10]:
view = visualize(        
    xyz_file = ab_xyz,
    sphere_radius = ab.GetSphereR() * 1.2,
    sphere_center = ab.GetSphereCenter(),
)
view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [22]:
ab_xyz = ""
ab.SetSphereR(4)
ab._freeze_molecule = None

In [23]:
ab.SetFreezeMol(0)

for _ in range(100):
    mol = ab.GetRandomGen().choice(ab.GetTotalMol())

    ab = ab.MoveMol(
        molecule=mol,
        max_step=4.0,
        max_rotation=30,
        max_closeness=1.0,
    )
    # print(ab.xyz)
    ab_xyz += ab.GetBlockXYZ()

# file_name = "cluster_test.xyz"
# with open(file_name, "w") as f:
#     f.write(ab_xyz)

# print(ab.freeze_molecule)
# print(ab.cluster_dictionary)

view = visualize(        
    xyz_file = ab_xyz,
    sphere_radius = ab.GetSphereR() * 1.2,
    sphere_center = ab.GetSphereCenter(),
)
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [24]:
import numpy as np

def overlapping(
    first_coordinates: list,
    second_coordinates: list,
    max_closeness: float = 1.0,
) -> bool:
    for first_atom in first_coordinates:
        for second_atom in second_coordinates:
            distance = np.linalg.norm(
                np.asarray(first_atom) - np.asarray(second_atom)
            )

            if distance < max_closeness:
                return True

    return False

In [36]:

def initialize_cluster(mol, max_closeness: float = 1.0) -> object:
    # center of mass coordinates
    sc_x = mol.GetSphereCenter()[0]
    sc_y = mol.GetSphereCenter()[1]
    sc_z = mol.GetSphereCenter()[2]

    # initializing a new cluster moving the first molecule
    # to the center of the cluster sphere
    molecule = mol.GetMol(0)
    new_cluster = Cluster(molecule).TranslateMol(0, sc_x, sc_y, sc_z)

    for i in range(1, mol.GetTotalMol()):
        # moving the next single molecule into the cluster sphere
        molecule = Cluster(mol.GetMol(i)).TranslateMol(0, sc_x, sc_y, sc_z)

        if overlapping(molecule.GetAtomicCoordinates(), new_cluster.GetAtomicCoordinates()):
            new_cluster += molecule
            new_cluster = new_cluster.MoveMol(
                i,
                max_step=None,
                max_rotation=None,
                max_closeness=max_closeness,
            )
        else:
            new_cluster += molecule

    return Cluster(
        new_cluster,
        freeze_molecule=mol.GetFreezeMol(),
        sphere_radius=mol.GetSphereR(),
        sphere_center=mol.GetSphereCenter(),
    )

In [27]:
hf6 = 6 * Cluster(hf)

hf6.SetSphereR(4)
hf6.SetSphereCenter((10, 10, 10))

print(hf6.GetSeed())

351546


In [37]:
hf6.SetSeed(932570)

new_hf6 =  initialize_cluster(hf6, max_closeness=2.0)
print(new_hf6.GetBlockXYZ())

12
charge: 0 multiplicity:1
H     	    10.00000000	    10.00000000	    10.00000000
F     	    11.00000000	    10.00000000	    10.00000000
H     	     7.97770358	    11.38395200	     8.75616546
F     	     8.91715949	    11.39121845	     8.41357273
H     	    10.93247453	     8.65279772	     8.04367022
F     	    11.90751396	     8.71839493	     7.83154971
H     	    12.34489203	     8.49903749	    10.98519686
F     	    13.27686053	     8.15031042	    10.88608007
H     	    12.26824384	    11.60105147	     7.42759545
F     	    13.22525369	    11.63622455	     7.13968052
H     	    13.93535314	     9.12938393	     7.24089639
F     	    14.85363218	     9.42949702	     7.49915157



In [39]:
new_view = visualize(
    xyz_file = new_hf6.GetBlockXYZ(),
    sphere_radius = new_hf6.GetSphereR() * 1.2,
    sphere_center = new_hf6.GetSphereCenter(),
)
new_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [40]:
water_molecule=[
    ("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)
    ]

w2 = Cluster(water_molecule, water_molecule)

# the water dimer was initialized with the same water molecule, so
# both molecules has the same coordinates an those atoms overlap each other
print(w2.GetBlockXYZ())

6
charge: 0 multiplicity:1
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000



In [41]:
#let's move any molecule randomly to avoid averlaping
new_w2 = w2.MoveMol(
    molecule = 1,
    max_step = 1.4,
    max_rotation = 30,
    max_closeness = 1.2,
)
print(new_w2.GetBlockXYZ())

6
charge: 0 multiplicity:1
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.79215445	     1.72021628	    -1.08097383
H     	     1.04955842	     2.54141496	    -0.65918512
H     	    -0.01309671	     2.04219097	    -0.67301225



In [45]:
#! Why eliminate one water molecule?
print(w2)

Cluster of (1) molecules and (6) total atoms
 #0: molecule with 3 atoms:
     --> atoms: [('O', 0.0, 0.0, 0.0), ('H', 0.58708, 0.75754, 0.0), ('H', -0.58708, 0.75754, 0.0)]
     --> charge: +0
     --> multiplicity: 1



In [25]:
from copy import deepcopy
from numpy import random
random_gen = random.default_rng(1234)

# freeze ibuprofen molecule to avoid be moved (just water molecules dancing)
# w2.freeze_molecule = 0

w2.sphere_radius = 4

new_w2 = deepcopy(w2)
new_w2 = new_w2.initialize_cluster(max_closeness=1)

# to save snapshot and show as a movie
w2_xyz = ""

total_steps = 100

for i in range(total_steps):
    # # molecule to be selected between [0, w.total_molecules]
    # # freeze molecules will not be moved or rotated
    mol = random_gen.choice(new_w2.total_molecules)

    print(new_w2.xyz)


    # saving coordinates as a string
    w2_xyz += new_w2.xyz
    # if i % 10 == 0: 
    #     w2_xyz += new_w2.xyz

    #----------------------------------------------------------

    new_w2 = new_w2.move_molecule(
        molecule=mol,
        max_step=2,
        max_rotation=30,
        max_closeness=0.5,
    )

# # file_name = "w2.xyz"
# # with open(file_name, "w") as f:
# #     f.write(w2_xyz)

	6
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	    -0.73062899	    -0.60344104	    -0.95359348
H     	     0.06088070	    -0.17836624	    -1.28729035
H     	    -0.99060035	     0.30794590	    -1.09613649

	6
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     1.16241837	    -1.66643581	    -2.92711679
H     	     1.69312182	    -0.94478375	    -3.26785412
H     	     0.53358734	    -1.09682266	    -3.37280970

	6
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.35891843	    -2.08866182	    -3.37382038
H     	

In [26]:
new_view = visualize(
    xyz_file = w2_xyz,
    sphere_radius =w2.sphere_radius * 1.2,
    sphere_center = w2.sphere_center,
)
new_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [27]:
# for several molecules we could move randomly one-by-one
# or we could just `initialize` this cluster with all its molecules 
# moved around avoiding overlapping
w6 = 3 * w2

# let's define the spherical boundary conditions
w6.sphere_radius = 10

# maximum closeness is the minimum distance for two atoms
new_w6 = w6.initialize_cluster(max_closeness=3)

print(new_w6.xyz)

	18
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	    -3.06363114	     2.35232138	     1.41415345
H     	    -2.70741365	     3.22843382	     1.56928525
H     	    -3.69112296	     2.76345474	     2.01060391
O     	     1.84007375	     1.69729582	     2.84351350
H     	     2.68551166	     2.00422357	     2.51251520
H     	     1.70851464	     2.63288076	     2.68257008
O     	    -3.57012303	    -1.95654771	    -1.17788784
H     	    -2.73711479	    -1.51528579	    -1.00492842
H     	    -3.78827963	    -1.02332087	    -1.18290230
O     	    -0.43178579	    -3.33563359	    -3.53577909
H     	    -0.35413792	    -2.48644592	    -3.09830808
H     	    -1.24873852	    -2.88168342	    -3.74802246
O     	     3.14349799	     2.78655547	    -3.08181605
H     	     3.26749236	     3.65116200	    -3.47629131
H     	     2.25994556	    

!pip install py3Dmol
import py3Dmol

In [30]:
new_view = visualize(
    xyz_file = new_w6.xyz,
    sphere_radius =new_w6.sphere_radius * 1.2,
    sphere_center = new_w6.sphere_center,
)
new_view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [34]:
from data.molecules_coordinates import water, ibuprofen 

# let's create a molecular object for the ibuprofen
ibu = Cluster(ibuprofen["atoms"])

# let's create a molecular object for one water molecule
w = Cluster(water["atoms"])

In [35]:
w6_overlap = Cluster(6 * w)

# let's initialize a cluster of six water without overlapping
w6 = Cluster(6 * w).initialize_cluster(max_closeness=2)


print(w6.xyz)


	18
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     1.78006070	    -1.01719227	    -2.09003717
H     	     2.25643012	    -0.24727421	    -2.40440660
H     	     1.16265524	    -0.29248887	    -1.97983169
O     	    -1.62872573	     0.47787098	     1.94646319
H     	    -1.54945055	     1.43219689	     1.90763940
H     	    -2.44199830	     0.80148377	     2.33683772
O     	    -1.62524234	     2.32937526	    -1.24642393
H     	    -0.91535761	     2.83621863	    -0.84930526
H     	    -2.02562982	     3.17281069	    -1.03000695
O     	     2.25592998	    -1.25644966	     0.66581252
H     	     3.01532295	    -0.87183256	     0.22544701
H     	     2.12357723	    -0.31040649	     0.74336789
O     	    -2.08006496	    -2.16673255	    -0.45239805
H     	    -1.52243254	    -1.52857269	    -0.00481805
H     	    -2.69533924	    

In [36]:
new_view = visualize(
    xyz_file = w6.xyz,
    sphere_radius = 5,
    sphere_center = w6.sphere_center,
)
new_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [37]:
ibu_w6 = Cluster(ibu + w6)

# let's define a sphere with center around the carboxylic acid group
ibu_w6.sphere_center = -3, 0.2, 0.2
ibu_w6.sphere_radius = 4

ibu_w6 = ibu_w6.initialize_cluster(max_closeness=2)

print(ibu_w6.xyz)

	50
-- charge=0 and multiplicity=1 --
C     	    -1.56098300	    -1.10947200	     1.17219000
C     	    -2.94477900	    -1.24360900	     1.33412100
C     	    -3.84450500	    -0.79651100	     0.35585200
C     	    -3.30141000	    -0.20160700	    -0.79830200
C     	    -1.92211300	    -0.07098900	    -0.95773900
C     	    -1.02102000	    -0.52007300	     0.02281900
H     	    -0.89290600	    -1.46731500	     1.95548700
H     	    -3.33425300	    -1.70043400	     2.24251900
H     	    -3.98572600	     0.18365200	    -1.54865500
H     	    -1.53569500	     0.39530700	    -1.86436000
C     	    -5.34102100	    -0.97417500	     0.53451400
H     	    -5.51739900	    -1.32551300	     1.55744700
C     	     0.47407100	    -0.34878900	    -0.15756300
H     	     0.76256300	    -0.69849900	    -1.16041200
H     	     1.00698600	    -0.99284600	     0.55851800
C     	    -5.90009800	    -2.02131200	    -0.45068600
H     	    -5.37923800	    -2.98385700	    -0.34592400
H     	    -6.96972400	    

In [39]:
new_view = visualize(
    xyz_file = ibu_w6.xyz,
    sphere_radius = ibu_w6.sphere_radius * 1.2,
    sphere_center = ibu_w6.sphere_center,
    style='stick',
)
new_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [40]:
from numpy import random
random_gen = random.default_rng(1234)

new_ibu_w6 = deepcopy(ibu_w6)

# to save snapshot and show as a movie
new_ibu_w6_xyz = ""

# freeze ibuprofen molecule to avoid be moved (just water molecules dancing)
# new_ibu_w6.freeze_molecule = 0

new_ibu_w6.sphere_radius = 3 
new_ibu_w6.sphere_center = -6.12410200, 0.37976300, 0.38520700


# to avoid overlapping and any molecule out of the sphere
ibu_w6 = ibu_w6.initialize_cluster(max_closeness=2)

total_steps = 100

for i in range(total_steps):
    # # molecule to be selected between [0, w.total_molecules]
    # # freeze molecules will not be moved or rotated
    mol = random_gen.choice(new_ibu_w6.total_molecules)

    if mol == 0:
        continue

    # saving coordinates as a string
    new_ibu_w6_xyz += new_ibu_w6.xyz
    # if i % 10 == 0: 
        # new_ibu_w6_xyz += new_ibu_w6.xyz

    # #----------------------------------------------------------

    new_ibu_w6 = new_ibu_w6.move_molecule(
        molecule=mol,
        max_closeness=0.5,
        max_step=2,
        max_rotation=30,    
    )

    #----------------------------------------------------------


    # printing current step
    print(
        f"\r progress {100.0*((i + 1)/total_steps):.2f}"
        f" % -- step {i + 1}/{total_steps}", end=''
        )
# -------------------------------------------------------
print("\n *** JOB DONE ***")
print(f"after {total_steps} steps\n")
# print(new_ibu_w6_xyz)

# open("new_ibu_w6.xyz", "w").write(new_ibu_w6_xyz)

 progress 100.00 % -- step 100/100
 *** JOB DONE ***
after 100 steps



In [41]:
new_view = visualize(
    xyz_file = new_ibu_w6_xyz,
    sphere_radius = new_ibu_w6.sphere_radius * 1.2,
    sphere_center = new_ibu_w6.sphere_center,
    style='stick',
)
new_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol